In [1]:
import arcpy

# Define the project directory and workspace
prj_dir = "C:/Users/samir/Documents/ArcGIS/Projects/multiLayerTest/multiLayerTest/"
arcpy.env.workspace = prj_dir + "multiLayerTest.gdb"

# Define input file and output feature class
in_file = "C:/Users/samir/Downloads/June Incident Log.csv"
out_feature_class = 'Events' #feature class has to be consistent with the rest of the code

# Define symbology layers
damage_symbology_layer = "C:/Users/samir/Downloads/incidentLogCategoriesLxyr/damage_or_destruction_of_buildings.lyrx"
military_symbology_layer = "C:/Users/samir/Downloads/incidentLogCategoriesLxyr/military_operations.lyrx"
noncombattant_symbology_layer = "C:/Users/samir/Downloads/incidentLogCategoriesLxyr/deliberate_targeting_of_noncombattant.lyrx"
weapons_symbology_layer = "C:/Users/samir/Downloads/incidentLogCategoriesLxyr/indiscriminate_use_of_weapons.lyrx"
killing_symbology_layer = "C:/Users/samir/Downloads/incidentLogCategoriesLxyr/willful_killing_of_civilians.lyrx"
detention_symbology_layer = "C:/Users/samir/Downloads/incidentLogCategoriesLxyr/unlawful_detention.lyrx"
dissapearance_symbology_layer = "C:/Users/samir/Downloads/incidentLogCategoriesLxyr/forced_disappearance.lyrx"


# Define shapefile path
shp_path = "C:/Users/samir/Downloads/sdn_adm_cbs_nic_ssa_20200831_shp/sdn_admbnda_adm1_cbs_nic_ssa_20200831.shp"

In [2]:
aprx_path = r"C:/Users/samir/Documents/ArcGIS/Projects/multiLayerTest/multiLayerTest.aprx"
aprx = arcpy.mp.ArcGISProject(aprx_path)
aprx = arcpy.mp.ArcGISProject("CURRENT")
map = aprx.listMaps()[0]  # assumes data to be added to first map listed
map.addDataFromPath(shp_path)

In [3]:
x_coords = "Longitude"
y_coords = "Latitude"
z_field = ''
arcpy.management.XYTableToPoint(in_file, out_feature_class, x_coords, y_coords, z_field, arcpy.SpatialReference(4326))
fields = ['State', 'Location']

with arcpy.da.UpdateCursor(out_feature_class, fields) as cursor:
    for row in cursor:
        if not row[1]:
            row[1] = row[0]
            cursor.updateRow(row)
            
# Load the feature layer for all points
all_points_layer = arcpy.MakeFeatureLayer_management(out_feature_class, "AllPointsLayer")

try:
    # Create a feature layer for damage points
    damage_layer = "Damage or Destruction of Buildings"
    damage_query = "IncidentType = 'Damage or destruction of civilian critical infrastructure' OR IncidentType = 'Damage or destruction, looting, or theft of cultural heritage?'" # If there's more than 1 value for the column
    arcpy.management.MakeFeatureLayer(all_points_layer, damage_layer, damage_query) 
    arcpy.ApplySymbologyFromLayer_management(damage_layer, damage_symbology_layer)
    mapx.addLayer(damage_layer)
except:
    pass

try:
    # Create a feature layer for military points
    military_layer = "Military Operations"
    military_query = "IncidentType = 'Military operations (battle, shelling)' OR IncidentType = 'Movement of military, paramilitary, or other troops and equipment'" # If there's more than 1 value for the column
    arcpy.management.MakeFeatureLayer(all_points_layer, military_layer, military_query)
    arcpy.ApplySymbologyFromLayer_management(military_layer, military_symbology_layer)
    mapx.addLayer(military_layer)
except:
    pass

try:
    # Create a feature layer for noncombattant points
    noncombattant_layer = "Deliberate Targeting of Noncombattant"
    noncombattant_query = "IncidentType = 'Deliberate targeting of noncombattant'" # If there's only 1 value for the column
    arcpy.management.MakeFeatureLayer(all_points_layer, noncombattant_layer, noncombattant_query)
    arcpy.ApplySymbologyFromLayer_management(noncombattant_layer, noncombattant_symbology_layer)
    mapx.addLayer(noncombattant_layer)
except:
    pass

try:
    # Create a feature layer for weapons points
    weapons_layer = "Indiscriminate Use of Weapons"
    weapons_query = "IncidentType = 'Indiscriminate use of weapons'"
    arcpy.management.MakeFeatureLayer(all_points_layer, weapons_layer, weapons_query)
    arcpy.ApplySymbologyFromLayer_management(weapons_layer, weapons_symbology_layer)
    mapx.addLayer(weapons_layer)
except:
    pass

try:
    # Create a feature layer for killing points
    killing_layer = "Willful Killing of Civilians"
    killing_query = "IncidentType = 'Willful killing of civilians'"
    arcpy.management.MakeFeatureLayer(all_points_layer, killing_layer, killing_query)
    arcpy.ApplySymbologyFromLayer_management(killing_layer, killing_symbology_layer)
    mapx.addLayer(killing_layer)
except:
    pass

try:
    # Create a feature layer for detention points
    detention_layer = "Unlawful Detention"
    detention_query = "IncidentType = 'Unlawful detention'"
    arcpy.management.MakeFeatureLayer(all_points_layer, detention_layer, detention_query)
    arcpy.ApplySymbologyFromLayer_management(detention_layer, detention_symbology_layer)
    mapx.addLayer(detention_layer)
except:
    pass

try:
    # Create a feature layer for dissapearance points
    dissapearance_layer = "Forced Disappearance"
    dissapearance_query = "IncidentType = 'Forced disappearance'"
    arcpy.management.MakeFeatureLayer(all_points_layer, dissapearance_layer, dissapearance_query)
    arcpy.ApplySymbologyFromLayer_management(dissapearance_layer, dissapearance_symbology_layer)
    mapx.addLayer(dissapearance_layer)
except:
    pass

aprx = arcpy.mp.ArcGISProject("CURRENT")
mapx = aprx.listMaps()[0]

lyr = map.listLayers('Events')[0] #feature class has to be consistent with the rest of the code
lyr.showLabels = True
lblClass = lyr.listLabelClasses()[0]
lblClass.expression = "$feature.Location"
l_cim = lyr.getDefinition("V2")
lc = l_cim.labelClasses[0]
lc.textSymbol.symbol.color = 'white'
lc.textSymbol.symbol.font = 8
lc.textSymbol.symbol.haloColor = 'black'
lc.textSymbol.symbol.haloSize = 1

lyr.setDefinition(l_cim)
lyr.showLabels = True

In [4]:
in_feature_class = "Events" #feature class has to be consistent with the rest of the code
method = "DBSCAN"
search_distance = "10000 Meters" # 10km
min_features_cluster = 2

# Add a new field to indicate cluster presence
field_name = "showMap"
field_type = "SHORT"
arcpy.AddField_management(in_feature_class, field_name, field_type)

try:
    # Run the DensityBasedClustering tool and capture the output feature class
    output_clusters = arcpy.stats.DensityBasedClustering(in_features=in_feature_class, cluster_method=method, search_distance=search_distance, min_features_cluster=min_features_cluster)
    print("Density-based clustering completed successfully.")

    # Initialize the new field with default value of 0
    with arcpy.da.UpdateCursor(in_feature_class, ["showMap"]) as cursor:
        for row in cursor:
            row[0] = 0  # Default value is 0
            cursor.updateRow(row)

    print(f"'showMap' field initialized with default value '0'.")

    # Update 'showMap' field based on the presence of clusters
    with arcpy.da.UpdateCursor(in_feature_class, ["OBJECTID", "showMap"]) as cursor:
        for row in cursor:
            if row[0] in [f[0] for f in arcpy.da.SearchCursor(output_clusters, ["OBJECTID"])]:
                row[1] = 1  # Set showMap to 1 if the point is part of a cluster
            cursor.updateRow(row)

    print("'showMap' field updated successfully.")

except arcpy.ExecuteError as e:
    print(f"ExecuteError: {e}")
except Exception as e:
    print(f"An error occurred: {e}")

# Assuming 'output_feature_class' is the correct output from DensityBasedClustering tool
output_feature_class = output_clusters[0]

Density-based clustering completed successfully.
'showMap' field initialized with default value '0'.
'showMap' field updated successfully.


In [5]:
cluster_field = "CLUSTER_ID"

try:
    unique_clusters = set([row[0] for row in arcpy.da.SearchCursor(output_feature_class, [cluster_field]) if row[0] != -1])
    print(f"Unique Clusters Identified: {unique_clusters}")
except RuntimeError as e:
    print(f"Error: {e}")

geodatabase_path = "C:/Users/samir/Documents/ArcGIS/Projects/multiLayerTest/multiLayerTest.gdb"
feature_class_name = "Events_DensityBasedClustering" #feature class has to be consistent with the rest of the code 
full_feature_class_path = f"{geodatabase_path}/{feature_class_name}"

Unique Clusters Identified: {1, 2, 3, 4, 5, 6}


In [6]:
import arcpy
import numpy as np

def MakeRec_LL(llx, lly, w, h):
    xyRecList = [[llx, lly], [llx, lly+h], [llx+w,lly+h], [llx+w,lly], [llx,lly]]
    array = arcpy.Array([arcpy.Point(*coords) for coords in xyRecList])
    rec = arcpy.Polygon(array)
    return rec

def _circle(radius, xc, yc, theta=1, clockwise=True):
    """Produce a circle/ellipse depending on parameters."""
    angles = np.deg2rad(np.arange(180.0, -180.0-theta, step=-theta)) if clockwise else np.deg2rad(np.arange(-180.0, 180.0+theta, step=theta))
    x_s = radius * np.cos(angles)
    y_s = radius * np.sin(angles)
    pnts = np.c_[x_s, y_s] + [xc, yc]
    return [arcpy.Point(*coords) for coords in pnts]

def create_layout_with_insets(aprx, map_name, title_text, unique_clusters, full_feature_class_path, damage_symbology_layer, out_feature_class):
    p = aprx
    lyt = p.createLayout(17, 11, 'INCH')
    m = p.listMaps(map_name)[0]

    map_frame_width, map_frame_height = 12, 8
    map_frame_x = (17 - map_frame_width) / 2
    map_frame_y = (11 - map_frame_height) / 2

    mf = lyt.createMapFrame(MakeRec_LL(map_frame_x, map_frame_y, map_frame_width, map_frame_height), m, "New Map Frame")
    
    
    #scalebar
    scale_bar_x = map_frame_x + (map_frame_width - 7) / 2
    scale_bar_y = map_frame_y - 1
    sbEnv = MakeRec_LL(scale_bar_x, scale_bar_y, 2.5, 0.3)
    sbName = 'Scale Line 1 Metric'
    sbStyle = p.listStyleItems('ArcGIS 2D', 'Scale_bar', sbName)[0]
    sb = lyt.createMapSurroundElement(sbEnv, 'Scale_bar', mf, sbStyle, 'New Scale Bar')
    sb.elementWidth = 3.5 # Adjust the width to make the scale bar larger
    
    #north arrow
    compass_x = scale_bar_x - 1  # Adjust the position to the left of the scale bar
    compass_y = scale_bar_y + 0.3
    naStyle = p.listStyleItems('ArcGIS 2D', 'North_Arrow', 'ArcGIS North 1')[0]
    na = lyt.createMapSurroundElement(arcpy.Point(compass_x, compass_y), 'North_Arrow', mf, naStyle, "Compass North Arrow")
    na.elementWidth = 0.3
    
    
    
    txtStyleItem = p.listStyleItems('ArcGIS 2D', 'TEXT', 'Title (Serif)')[0]
    ptTxt = p.createTextElement(lyt, arcpy.Point(11, 10), 'POINT', title_text, 6, style_item=txtStyleItem)
    ptTxt.setAnchor('Center_Point')
    ptTxt.elementPositionX = 8.5
    ptTxt.elementPositionY = 10
    ptTxt.textSize = 28
    ptTxt.fontFamilyName = "Tahoma"

    
    
    #legend
    """legSi = p.listStyleItems('ArcGIS 2D', 'LEGEND', 'Legend 3')[0]
    legend = lyt.createMapSurroundElement(arcpy.Point(9.87, 1.43), 'LEGEND', mf, legSi, 'New Legend Element')
    legend.elementWidth = 7
    legend.elementHeight = 3
    legend.fittingStrategy = 'AdjustFontSize'
    legend.columnCount = 5
    legend.title = title_text
    legend.syncLayerVisibility = True"""

    inset_radius = 1.4
    inset_x = 15
    inset_y = 4
    
    all_points_layer = arcpy.MakeFeatureLayer_management(out_feature_class, "AllPointsLayer")
    for cluster_id in unique_clusters:
        cluster_layer_name = f"Cluster_{cluster_id}_Layer"
        cluster_query = f"CLUSTER_ID = {cluster_id}"
        cluster_layer = arcpy.MakeFeatureLayer_management(full_feature_class_path, cluster_layer_name, cluster_query).getOutput(0)

        inset_map_name = f"Inset_Map_{cluster_id}"
        inset_map = aprx.createMap(inset_map_name)
        inset_map.addLayer(cluster_layer)

            # Create a filtered layer for the hospital data points that match the cluster
            # Assuming CLUSTER_ID is numeric
        try:
            # Create a feature layer for damage points
            damage_inset = arcpy.management.MakeFeatureLayer(all_points_layer, damage_layer, damage_query).getOutput(0)
            arcpy.ApplySymbologyFromLayer_management(damage_inset, damage_symbology_layer)
            inset_map.addLayer(damage_inset)
        except:
            pass
            
        try:
            # Create a feature layer for military points
            military_inset = arcpy.management.MakeFeatureLayer(all_points_layer, military_layer, military_query).getOutput(0)
            arcpy.ApplySymbologyFromLayer_management(military_inset, military_symbology_layer)
            inset_map.addLayer(military_inset)
        except:
            pass

        try:
            # Create a feature layer for noncombattant points
            noncombattant_inset = arcpy.management.MakeFeatureLayer(all_points_layer, noncombattant_layer, noncombattant_query).getOutput(0)
            arcpy.ApplySymbologyFromLayer_management(noncombattant_inset, noncombattant_symbology_layer)
            inset_map.addLayer(noncombattant_inset)
        except:
            pass

        try:
            # Create a feature layer for weapons points
            weapons_inset = arcpy.management.MakeFeatureLayer(all_points_layer, weapons_layer, weapons_query).getOutput(0)
            arcpy.ApplySymbologyFromLayer_management(weapons_inset, weapons_symbology_layer)
            inset_map.addLayer(weapons_inset)
        except:
            pass

        try:
            # Create a feature layer for killing points
            killing_inset = arcpy.management.MakeFeatureLayer(all_points_layer, killing_layer, killing_query).getOutput(0)
            arcpy.ApplySymbologyFromLayer_management(killing_inset, killing_symbology_layer)
            inset_map.addLayer(killing_inset)
        except:
            pass

        try:
            # Create a feature layer for detention points
            detention_inset = arcpy.management.MakeFeatureLayer(all_points_layer, detention_layer, detention_query).getOutput(0)
            arcpy.ApplySymbologyFromLayer_management(detention_inset, detention_symbology_layer)
            inset_map.addLayer(detention_inset)
        except:
            pass

        try:
            # Create a feature layer for dissapearance points
            dissapearance_inset = arcpy.management.MakeFeatureLayer(all_points_layer, dissapearance_layer, dissapearance_query).getOutput(0)
            arcpy.ApplySymbologyFromLayer_management(dissapearance_inset, dissapearance_symbology_layer)
            inset_map.addLayer(dissapearance_inset)
        except:
            pass


        circle_coords = _circle(inset_radius, inset_x, inset_y)
        inset_circle = arcpy.Polygon(arcpy.Array(circle_coords))
        inset_map_frame = lyt.createMapFrame(inset_circle, inset_map, f"Cluster_{cluster_id}_MapFrame")

        inset_y += 2 * inset_radius + 0.5
        if inset_y + 2 * inset_radius > 17:
            inset_y = 1
            inset_x += 2 * inset_radius + 0.5


    lyt.openView()
    return lyt

# Define parameters and call the function
title_text = "Automatic Mapping"
lyt = create_layout_with_insets(aprx, map.name, title_text, unique_clusters, full_feature_class_path, damage_symbology_layer, out_feature_class)

In [7]:
output_png = f"C:/Users/samir/Documents/ArcGIS/Projects/multiLayerTest/Map"
lyt.exportToPNG(output_png, resolution=200)

print(f"Layout exported to PNG: {output_png}")

Layout exported to PNG: C:/Users/samir/Documents/ArcGIS/Projects/multiLayerTest/Map
